<a href="https://colab.research.google.com/github/Chayansp/MyProject/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Extract data from CSV

In [3]:
import pandas as pd

file = "/content/SalesTransaction.csv"
sales_df = pd.read_csv(file)

type(sales_df)

pandas.core.frame.DataFrame

In [4]:
sales_df.head()

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom
3,581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom
4,581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom


In [5]:
print(f"number of rows: {len(sales_df)}")

number of rows: 536350


#Extraction data from REST API


In [34]:
import requests

url = 'https://de-training-2020-7au6fmnprq-de.a.run.app/currency_gbp/all'
response = requests.get(url)
response.status_code
pound_rate = response.json()

**Convert json to DataFrame**

In [35]:
conversion_rate = pd.DataFrame(pound_rate)
conversion_rate

,Rate
2018-01-01T00:00:00.000Z,43.9991
2018-01-02T00:00:00.000Z,44.0731
2018-01-03T00:00:00.000Z,43.7025
2018-01-04T00:00:00.000Z,43.6914
2018-01-05T00:00:00.000Z,43.6490
...,...
2020-02-15T00:00:00.000Z,40.7223
2020-02-16T00:00:00.000Z,40.6612
2020-02-17T00:00:00.000Z,40.5658
2020-02-18T00:00:00.000Z,40.5582


**Reset index and rename**

In [8]:
conversion_rate = conversion_rate.reset_index().rename(columns={"index": "date"})
conversion_rate[:4]

,date,Rate
0,2018-01-01T00:00:00.000Z,43.9991
1,2018-01-02T00:00:00.000Z,44.0731
2,2018-01-03T00:00:00.000Z,43.7025
3,2018-01-04T00:00:00.000Z,43.6914


# Join transaction from MySQL database and conversion rate from REST API

In [9]:
#duplicate Date column then change Date column name to date column name
sales_df['date'] = sales_df['Date']
sales_df.head(3)

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country,date
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom,12/9/2019
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom,12/9/2019
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom,12/9/2019


In [10]:
#convert date in both sales_df and conversion_rate to same format
sales_df['date'] = pd.to_datetime(sales_df['date']).dt.date
conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date
sales_df

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country,date
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom,2019-12-09
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom,2019-12-09
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom,2019-12-09
3,581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom,2019-12-09
4,581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom,2019-12-09
...,...,...,...,...,...,...,...,...,...
536345,C536548,12/1/2018,22168,Organiser Wood Antique White,18.96,-2,12472.0,Germany,2018-12-01
536346,C536548,12/1/2018,21218,Red Spotty Biscuit Tin,14.09,-3,12472.0,Germany,2018-12-01
536347,C536548,12/1/2018,20957,Porcelain Hanging Bell Small,11.74,-1,12472.0,Germany,2018-12-01
536348,C536548,12/1/2018,22580,Advent Calendar Gingham Sack,16.35,-4,12472.0,Germany,2018-12-01


In [11]:
#merge sales_df and conversion_rate date by left join with date key
final_df = sales_df.merge(conversion_rate, how="left", left_on="date", right_on="date")
final_df.head(3)

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country,date,Rate
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom,2019-12-09,39.8391
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom,2019-12-09,39.8391
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom,2019-12-09,39.8391


In [24]:
#convert to float type
final_df["Price"] = final_df["Price"].astype(float)

In [28]:
#multiple between 'Price' and 'Rate' to 'THBPrice'
def convert_rate(price, rate):
  return price * rate
##.apply()
final_df["THBPrice"] = final_df.apply(lambda x: convert_rate(x["Price"],x["Rate"]),axis=1)

"""
another way to create 'THBPrice' from multiple between 'Price' and 'Rate'
final_df["THBPrice"] = final_df["Price"]*final_df["Rate"]
"""


'\nanother way to create \'THBPrice\' from multiple between \'Price\' and \'Rate\'\nfinal_df["THBPrice"] = final_df["Price"]*final_df["Rate"]\n'

In [29]:
final_df

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country,Rate,THBPrice
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom,39.8391,855.345477
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom,39.8391,424.286415
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom,39.8391,459.344823
3,581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom,39.8391,424.286415
4,581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom,39.8391,475.678854
...,...,...,...,...,...,...,...,...,...,...
536345,C536548,12/1/2018,22168,Organiser Wood Antique White,18.96,-2,12472.0,Germany,42.0373,797.027208
536346,C536548,12/1/2018,21218,Red Spotty Biscuit Tin,14.09,-3,12472.0,Germany,42.0373,592.305557
536347,C536548,12/1/2018,20957,Porcelain Hanging Bell Small,11.74,-1,12472.0,Germany,42.0373,493.517902
536348,C536548,12/1/2018,22580,Advent Calendar Gingham Sack,16.35,-4,12472.0,Germany,42.0373,687.309855


In [19]:
#drop date
final_df = final_df.drop("date", axis=1)

In [ ]:
#save to .csv file
final_df.to_csv("output.csv",index=False)